<a href="https://colab.research.google.com/github/irenezi/solid-guide/blob/master/coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import time
from pandas import read_csv, set_option
from numpy import set_printoptions
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, RFE
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import precision_score, confusion_matrix
from sklearn.ensemble import ExtraTreesClassifier
from keras.layers import Dense, Input 
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.regularizers import l1
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
def filter_constant_columns(df_data_set_x, variance=0):
  # This function filters out all the constant columns (variance = 0)
  
  constant_filter = VarianceThreshold(threshold=variance)
  constant_filter.fit(df_data_set_x)
  non_constant = constant_filter.get_support()

  columns_to_keep = (df_data_set_x.columns[constant_filter.get_support()])

  return df_data_set_x[columns_to_keep]

In [ ]:
def normalize(x_train_input, x_test_input):
  # Normalize train and test data
  # Normalize the training set and save the normalization parameters
  # Normalize the test set using the training normalization parameters

  scaler=Normalizer().fit(x_train_input)
  x_train_norm=scaler.transform(x_train_input)
  x_test_norm=scaler.transform(x_test_input)
  
  return x_train_norm, x_test_norm

In [ ]:
def rescale(x_train_input, x_test_input):
  # Rescale train and test data
  # Rescale the training set and save the parameters
  # Rescale the test set using the training parameters

  scaler=MinMaxScaler(feature_range=(0,1))
  scaler = scaler.fit(x_train_input)
  x_train_resc = scaler.transform(x_train_input)
  x_test_resc = scaler.transform(x_test_input)
  return x_train_resc, x_test_resc

In [ ]:
def compare_algorithms(x_train_input, y_train_output, x_test_input, y_test_output):

  # prints the validation accuracy (using K-Folf method on the train data set)
  # and the test accuracy by apllying a variety of models to the test data set  
  
  models=[]
  models.append(('LR', LogisticRegression(solver='liblinear'))) 
  models.append(('LDA', LinearDiscriminantAnalysis()))
  models.append(('KNN', KNeighborsClassifier()))
  #models.append(('CART', DecisionTreeClassifier()))
  #models.append(('NB', GaussianNB()))
  models.append(('SVC', SVC()))
  models.append(('LnSVC', LinearSVC()))

  for name, model in models:
    model.fit(x_train_input, y_train_output)
    train_prediction = model.predict(x_train_input)
    train_acc = accuracy_score(y_train_output, train_prediction)
    test_prediction = model.predict(x_test_input)
    test_acc = accuracy_score(y_test_output, test_prediction)
    print("%s: train_acc: %.3f / test_acc: %.3f" % (name,train_acc, test_acc))
  

In [ ]:
def kbest(x_train_input, y_train_output, x_test_input):
  # feature selection using K-best
  
  # apply SelectKBest class to extract top 10 best features
  bestfeatures = SelectKBest(score_func=chi2, k=10)
  fit = bestfeatures.fit(x_train_input, y_train_output)
  dfscores = pd.DataFrame(fit.scores_)
  df = pd.DataFrame(x_train_input)
  dfcolumns = pd.DataFrame(df.columns)

  # concat two dataframes for better visualization 
  featureScores = pd.concat([dfcolumns,dfscores],axis=1)
  # naming the dataframe columns
  featureScores.columns = ['Specs','Score']  

  # print 10 best features
  print(featureScores.nlargest(10,'Score'))  

  # find the columns of the 10 best features
  featureScores=featureScores.nlargest(10,'Score').values
  col=list(featureScores[:,0])

  # returned train data set
  features = fit.transform(x_train_input)

  # returned test data set
  df_test_features = pd.DataFrame(x_test_input)
  test_features = df_test_features[col]

  return features, test_features.values

In [ ]:
def rfe_elimination(model, x_train_input, y_train_output, x_test_input):
  # apply Recursive Feature Elimination(RFE) for feature selection

  rfe=RFE(model,10)
  fit=rfe.fit(x_train_input, y_train_output)
  #print("Num Features: %d" % fit.n_features_)
  #print("Selected Features: %s" % fit.support_)
  #print("Feature Ranking: %s" % fit.ranking_)

  dfscores = pd.DataFrame(fit.ranking_)
  df = pd.DataFrame(x_train_input)
  dfcolumns = pd.DataFrame(df.columns)

  # concat two dataframes for better visualization 
  featureScores = pd.concat([dfcolumns,dfscores],axis=1)
  # naming the dataframe columns
  featureScores.columns = ['Specs','Score']  
  print(featureScores.nsmallest(10,'Score')) 

  # find the columns of the 10 best features
  featureScores=(featureScores.nsmallest(10,'Score')).values
  col=list(featureScores[:,0])

  # returned train data set
  features=fit.transform(x_train_input)

  # returned test data set
  df_test_features = pd.DataFrame(x_test_input)
  test_features = df_test_features[col]

  return features, test_features.values

In [ ]:
def extra_trees_classifier(x_train_input, y_train_output, x_test_input):
  # feature selection with extra tree classifier - Feature Importance

  model = ExtraTreesClassifier(random_state=21)
  tree=model.fit(x_train_input,y_train_output)
  #print(model.feature_importances_)

  dfscores = pd.DataFrame(tree.feature_importances_)
  df = pd.DataFrame(x_train_input)
  dfcolumns = pd.DataFrame(df.columns)
  
  # concat two dataframes for better visualization 
  featureScores = pd.concat([dfcolumns,dfscores],axis=1)
  # naming the dataframe columns
  featureScores.columns = ['Specs','Score']  
  featureScores=featureScores.nlargest(10,'Score')
  print(featureScores)

  # find the columns of the 10 best features
  featureScores=featureScores.values
  col=list(featureScores[:,0])

  # returned train data set
  features=df[col]

  # returned test data set
  df_test_features = pd.DataFrame(x_test_input)
  test_features = df_test_features[col]

  return features.values, test_features.values

In [ ]:
#no need to reshape!!!!
#reshape
#x_train_norm = x_train_norm.reshape((len(x_train_norm), np.prod(x_train_norm.shape[1:]))) 
#x_test_norm = x_test_norm.reshape((len(x_test_norm), np.prod(x_test_norm.shape[1:])))
#print(x_train_norm.shape)

In [ ]:
def stacked_autoencoder(x_train_input, x_test_input):
  # returns the encoder model of a stacked autoencoder with 2 hidden layers

  input = Input(shape=(82,))
  hidden_1 = Dense(500, activation='relu')(input)
  hidden_2 = Dense(300, activation='relu')(hidden_1)
  code = Dense(20, activation='relu')(hidden_2)
  hidden_3 = Dense(300, activation='relu')(code)
  hidden_4 = Dense(500, activation='relu')(hidden_3) 
  output = Dense(82, activation='sigmoid')(hidden_4)

  # define autoencoder model
  autoencoder = Model(inputs=input, outputs=output)
  # compile model
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
  # fit model to reconstruct the input
  autoencoder.fit(x_train_input, x_train_input, batch_size=200, epochs=20, validation_data=(x_test_input, x_test_input))

  # define encoder model
  encoder1 = Model(inputs=input, outputs=code)
#  # save the encoder model to file
#  encoder.save("encoder1.h5")

  return encoder1

In [ ]:
def denoising_autoencoder(x_train_input, x_test_input):
  # returns the encoder model of a denoising autoencoder with 2 hidden layers

  # add noise
  noise_factor = 0.4
  x_train_noisy = x_train_input + noise_factor * np.random.normal(size=x_train_input.shape)
  x_test_noisy = x_test_input + noise_factor * np.random.normal(size=x_test_input.shape)
  x_train_noisy = np.clip(x_train_noisy, 0.0, 1.0)
  x_test_noisy = np.clip(x_test_noisy, 0.0, 1.0)

  input = Input(shape=(82,))
  hidden_1 = Dense(500, activation='relu')(input)
  hidden_2 = Dense(300, activation='relu')(hidden_1)
  code = Dense(20, activation='relu')(hidden_2)
  hidden_3 = Dense(300, activation='relu')(code)
  hidden_4 = Dense(500, activation='relu')(hidden_3) 
  output = Dense(82, activation='sigmoid')(hidden_4)

  #define noisy aytoencoder model
  autoencoder_noisy = Model(inputs=input, outputs=output)
  # compile the model
  autoencoder_noisy.compile(optimizer='adam', loss='binary_crossentropy')
  # fit model to reconstruct the input
  autoencoder_noisy.fit(x_train_noisy, x_train_noisy, batch_size=200, epochs=20, validation_data=(x_test_noisy, x_test_noisy))

  # define the encoder model
  encoder2 = Model(inputs=input, outputs=code)
  #  # save the encoder model to file
  #  encoder.save("encoder2.h5")

  return encoder2

In [ ]:
def sparse_autoencoder(x_train_input, x_test_input):
  #returns the encoder model of a sparse autoencoder 

  input_size = 82
  code_size = 20

  input = Input(shape=(input_size,))
  code = Dense(code_size,activity_regularizer=l1(10e-6), activation='relu')(input) 
  output = Dense(input_size, activation='sigmoid')(code)

  # define autoencoder model
  autoencoder_regularized = Model(inputs=input, outputs=output) 
  # compile the autoencoder model
  autoencoder_regularized.compile(optimizer='adam', loss='binary_crossentropy') 
  # fit autoencoder model to reconstruct the input
  autoencoder_regularized.fit(x_train_input, x_train_input, batch_size=200,epochs=20, validation_data=(x_test_input, x_test_input))
  
  # define the encoder model
  encoder3 = Model(input, code)
  #  # save the encoder model to file
  #  encoder.save("encoder3.h5")

  return encoder3

In [ ]:
def encoder_transformation(encoder, x_train_input, x_test_input):
  # use the encoder models to transform the data

  # train_data transformation
  x_train_encoded = encoder.predict(x_train_input)
  # test data transformation
  x_test_encoded = encoder.predict(x_test_input)

  return x_train_encoded, x_test_encoded

In [ ]:
def mixed_data(train1, train2, test1, test2):
  # concatenates columns wise 2 numpy arrays
  # in order to combine original and extracted data

  train_array = np.concatenate((train1, train2), axis=1)
  test_array = np.concatenate((test1, test2), axis=1)

  return train_array, test_array

In [ ]:
def compare_algorithms_2(x_train_input, y_train_output, x_test_input, y_test_output):

  # applying a selection of models from compare_algorithms method
  # prints the validation accuracy (using K-Folf method on the train data set)
  # and the test accuracy, AUC metric, classification report and 
  # time to built (TBM) and to test (TTM) the model 

  models=[]
  models.append(('LR', LogisticRegression(solver='liblinear'))) 
  models.append(('LDA', LinearDiscriminantAnalysis()))
  models.append(('LnSVC', LinearSVC()))
  
  for name, model in models:

    start = time.time()
    model.fit(x_train_input, y_train_output)
    train_time = time.time()

    test_prediction = model.predict(x_test_input)
    matrix = confusion_matrix(y_test_output, test_prediction)
    acc = round(accuracy_score(y_test_output, test_prediction),3)
    DR = round(recall_score(y_test_output, test_prediction),3)
    Prec = round(precision_score(y_test_output, test_prediction),3)
    FAR = round(float(matrix[0][1] / (matrix[0][1] + matrix[0][0])),3)
    FNR = round(float(matrix[1][0] / (matrix[1][0] + matrix[1][1])),3)
    Type_1 = matrix[0][1]
    Type_2 = matrix[1][0]
    
    test_time = time.time()
    
    print("%s:" % (name))
    print(matrix)
    print("acc: ",acc," / DR: ",DR," / Prec: ",Prec," / FAR: ",FAR," / FNR: ",FNR)
    print("Type I error: ",Type_1, " / Type II error: ",Type_2)  
    print("TBM: ",round(train_time-start,2))
    print("TTM: ",round(test_time-train_time,2), "\n")


In [ ]:
def mixed_data_2(x_train_input, y_train_output):
  # concatenates columns wise 2 numpy arrays
  # to create a data set (dataframe) that contains both x and y variables

  train_array = np.concatenate((x_train_input, y_train_output.reshape(-1,1)), axis=1)
  df_train_array = pd.DataFrame(train_array)
  
  return df_train_array

In [ ]:
def reduced_data(df_data, new_size=600):
  # creates a reduced dataset in order to minimize time needed for grid search

  number_of_rows = df_data.shape[0]
  random_indices = np.random.choice(number_of_rows, size=new_size, replace=False)
  random_rows = df_data.values[random_indices, :]

  X=random_rows[:,0:(len(df_data.columns)-1)]
  Y=random_rows[:,(len(df_data.columns)-1)]

  return X,Y

RUN

DEALING WITH THE DATA

In [ ]:
#load the data
filename1 ="train_imperson_without4n7_balanced_data.csv"
filename2 ="test_imperson_without4n7_balanced_data.csv"

df_train_data = read_csv(filename1)
df_test_data = read_csv(filename2)


# apply constant column filter

# not matched constant columns in the train and test data set

# merge them to one data set
df_full_data = df_train_data.append(df_test_data)

#apply the filter
df_full_data_filtered = filter_constant_columns(df_full_data)

# separate the data sets
df_train_data_filtered = df_full_data_filtered[0:97044]
df_test_data_filtered = df_full_data_filtered[97044:]


#separate the filtered x,y
x_train = df_train_data_filtered[df_train_data_filtered.columns[:-1]].values
y_train = df_train_data_filtered[df_train_data_filtered.columns[-1]].values
x_test = df_test_data_filtered[df_test_data_filtered.columns[:-1]].values
y_test = df_test_data_filtered[df_test_data_filtered.columns[-1]].values

In [ ]:
x_train_norm, x_test_norm = normalize(x_train, x_test)

In [ ]:
x_train_resc, x_test_resc = rescale(x_train, x_test)

FITTING MODELS TO COMPARE, USING: Raw data / Normalized data / Rescaled data

In [ ]:
# fitting models to raw data
compare_algorithms(x_train, y_train, x_test, y_test)

LR: train_acc: 0.992 / test_acc: 0.868
LDA: train_acc: 0.985 / test_acc: 0.931
KNN: train_acc: 0.999 / test_acc: 0.534
SVC: train_acc: 0.995 / test_acc: 0.530
LnSVC: train_acc: 0.995 / test_acc: 0.652


In [ ]:
# fitting models to normalized data
compare_algorithms(x_train_norm, y_train, x_test_norm, y_test)

LR: train_acc: 0.989 / test_acc: 0.965
LDA: train_acc: 0.985 / test_acc: 0.614
KNN: train_acc: 0.999 / test_acc: 0.532
SVC: train_acc: 0.995 / test_acc: 0.531
LnSVC: train_acc: 0.992 / test_acc: 0.525


In [ ]:
# fitting models to rescaled data
compare_algorithms(x_train_resc, y_train, x_test_resc, y_test)

LR: train_acc: 0.996 / test_acc: 0.506
LDA: train_acc: 0.985 / test_acc: 0.931
KNN: train_acc: 1.000 / test_acc: 0.536
SVC: train_acc: 0.997 / test_acc: 0.499
LnSVC: train_acc: 0.997 / test_acc: 0.509


FEATURE SELECTION (TOP 10 FEATURES) USING: K-BEST / RFE / EXTRA TREES

In [ ]:
# K-Best method for raw data
kbest_train_features, kbest_test_features = kbest(x_train, y_train, x_test)

    Specs         Score
26     26  32124.682195
16     16  28936.000138
22     22  28321.490224
13     13  24346.021809
81     81  13311.733744
15     15  12317.666392
2       2  12076.290588
3       3  12076.290588
23     23   7911.296191
36     36   7720.479234


In [ ]:
# K-Best method for normalized data
kbest_train_features_norm, kbest_test_features_norm = kbest(x_train_norm, y_train, x_test_norm)

    Specs        Score
26     26  7185.552254
22     22  6458.853639
16     16  6265.412604
13     13  5162.369647
81     81  2648.775398
15     15  2569.185754
2       2  2378.606418
3       3  2378.606418
23     23  2091.641720
28     28  1758.070006


In [ ]:
# K-Best method for rescaled data
kbest_train_features_resc, kbest_test_features_resc = kbest(x_train_resc, y_train, x_test_resc)

    Specs         Score
26     26  32124.682195
16     16  28936.000138
22     22  28321.490224
13     13  24346.021809
81     81  13311.733744
15     15  12317.666392
2       2  12076.290588
3       3  12076.290588
23     23   7911.296191
36     36   7720.479234


In [ ]:
# Recursive Feature Elimination method for raw data
rfe_train_features, rfe_test_features = rfe_elimination(LogisticRegression(solver='liblinear'), x_train, y_train, x_test)

    Specs  Score
3       3      1
13     13      1
22     22      1
25     25      1
36     36      1
44     44      1
46     46      1
50     50      1
55     55      1
74     74      1


In [ ]:
# Recursive Feature Elimination method for normalized data
rfe_train_features_norm, rfe_test_features_norm = rfe_elimination(LogisticRegression(solver='liblinear'), x_train_norm, y_train, x_test_norm)

    Specs  Score
13     13      1
15     15      1
18     18      1
22     22      1
26     26      1
36     36      1
44     44      1
46     46      1
69     69      1
73     73      1


In [ ]:
# Recursive Feature Elimination method for rescaled data
rfe_train_features_resc, rfe_test_features_resc = rfe_elimination(LogisticRegression(solver='liblinear'), x_train_resc, y_train, x_test_resc)

    Specs  Score
13     13      1
20     20      1
23     23      1
25     25      1
36     36      1
50     50      1
51     51      1
55     55      1
62     62      1
74     74      1


In [ ]:
# Extra Trees Classifier method for raw data
extree_train_features, extree_test_features = extra_trees_classifier(x_train, y_train, x_test)

    Specs     Score
15     15  0.109607
26     26  0.105623
22     22  0.098700
13     13  0.068557
16     16  0.062296
23     23  0.060353
28     28  0.052966
11     11  0.051659
36     36  0.047049
21     21  0.032328


In [ ]:
# Extra Trees Classifier method for normalized data
extree_train_features_norm, extree_test_features_norm = extra_trees_classifier(x_train_norm, y_train, x_test_norm)

    Specs     Score
26     26  0.100260
15     15  0.088078
22     22  0.073067
16     16  0.070049
13     13  0.050870
28     28  0.048229
36     36  0.036058
75     75  0.033650
23     23  0.033624
11     11  0.031078


In [ ]:
# Extra Trees Classifier method for rescaled data
extree_train_features_resc, extree_test_features_resc = extra_trees_classifier(x_train_resc, y_train, x_test_resc)

    Specs     Score
15     15  0.109607
26     26  0.105623
22     22  0.098700
13     13  0.068557
16     16  0.062296
23     23  0.060353
28     28  0.052966
11     11  0.051659
36     36  0.047049
21     21  0.032328


MODEL EVALUATION WITH SELECTED FEATURES

In [ ]:
# raw data

In [ ]:
# compare models using top 10 features from K-Best for raw data
compare_algorithms(kbest_train_features, y_train, kbest_test_features, y_test)

LR: train_acc: 0.961 / test_acc: 0.500
LDA: train_acc: 0.929 / test_acc: 0.277
KNN: train_acc: 0.999 / test_acc: 0.500
SVC: train_acc: 0.992 / test_acc: 0.500
LnSVC: train_acc: 0.960 / test_acc: 0.500


In [ ]:
# compare models using top 10 features from RFE for raw data
compare_algorithms(rfe_train_features, y_train, rfe_test_features, y_test)

LR: train_acc: 0.985 / test_acc: 0.861
LDA: train_acc: 0.969 / test_acc: 0.820
KNN: train_acc: 1.000 / test_acc: 0.531
SVC: train_acc: 0.997 / test_acc: 0.521
LnSVC: train_acc: 0.993 / test_acc: 0.866


In [ ]:
# compare models using top 10 features from Extra Trees for raw data
compare_algorithms(extree_train_features, y_train, extree_test_features, y_test)

LR: train_acc: 0.970 / test_acc: 0.959
LDA: train_acc: 0.938 / test_acc: 0.916
KNN: train_acc: 0.998 / test_acc: 0.521
SVC: train_acc: 0.988 / test_acc: 0.502
LnSVC: train_acc: 0.971 / test_acc: 0.958


In [ ]:
# normalized data

In [ ]:
# compare models using top 10 features from K-Best for normalized data
compare_algorithms(kbest_train_features_norm, y_train, kbest_test_features_norm, y_test)

LR: train_acc: 0.929 / test_acc: 0.500
LDA: train_acc: 0.928 / test_acc: 0.675
KNN: train_acc: 0.998 / test_acc: 0.500
SVC: train_acc: 0.982 / test_acc: 0.500
LnSVC: train_acc: 0.941 / test_acc: 0.499


In [ ]:
# compare models using top 10 features from RFE for normalized data
compare_algorithms(rfe_train_features_norm, y_train, rfe_test_features_norm, y_test)

LR: train_acc: 0.987 / test_acc: 0.520
LDA: train_acc: 0.974 / test_acc: 0.958
KNN: train_acc: 0.999 / test_acc: 0.532
SVC: train_acc: 0.992 / test_acc: 0.527
LnSVC: train_acc: 0.988 / test_acc: 0.519


In [ ]:
# compare models using top 10 features from Extra Trees for normalized data
compare_algorithms(extree_train_features_norm, y_train, extree_test_features_norm, y_test)

LR: train_acc: 0.954 / test_acc: 0.964
LDA: train_acc: 0.934 / test_acc: 0.907
KNN: train_acc: 0.999 / test_acc: 0.528
SVC: train_acc: 0.989 / test_acc: 0.507
LnSVC: train_acc: 0.962 / test_acc: 0.967


In [ ]:
# rescaled data

In [ ]:
# compare models using top 10 features from K-Best for rescaled data
compare_algorithms(kbest_train_features_resc, y_train, kbest_test_features_resc, y_test)

LR: train_acc: 0.961 / test_acc: 0.500
LDA: train_acc: 0.929 / test_acc: 0.277
KNN: train_acc: 0.999 / test_acc: 0.500
SVC: train_acc: 0.992 / test_acc: 0.500
LnSVC: train_acc: 0.960 / test_acc: 0.500


In [ ]:
# compare models using top 10 features from RFE for rescaled data
compare_algorithms(rfe_train_features_resc, y_train, rfe_test_features_resc, y_test)

LR: train_acc: 0.992 / test_acc: 0.813
LDA: train_acc: 0.980 / test_acc: 0.783
KNN: train_acc: 0.999 / test_acc: 0.535
SVC: train_acc: 0.997 / test_acc: 0.494
LnSVC: train_acc: 0.995 / test_acc: 0.816


In [ ]:
# compare models using top 10 features from Extra Trees for rescaled data
compare_algorithms(extree_train_features_resc, y_train, extree_test_features_resc, y_test)

LR: train_acc: 0.969 / test_acc: 0.959
LDA: train_acc: 0.938 / test_acc: 0.916
KNN: train_acc: 0.998 / test_acc: 0.521
SVC: train_acc: 0.988 / test_acc: 0.502
LnSVC: train_acc: 0.971 / test_acc: 0.958


USE OF AUTOENCODERS FOR FEATURE EXTRACTION: STACKED / DENOISING / SPARSE

---




In [ ]:
# creating the encoder model of the stacked autoencoder
# for the raw, normalized and rescaled data

encoder1_raw = stacked_autoencoder(x_train, x_test)
encoder1_norm = stacked_autoencoder(x_train_norm, x_test_norm)
encoder1_resc = stacked_autoencoder(x_train_resc, x_test_resc)


Epoch 1/20
486/486 [==============================] - 10s 19ms/step - loss: 0.1386 - val_loss: 0.5030
Epoch 2/20
486/486 [==============================] - 9s 18ms/step - loss: 0.0604 - val_loss: 0.4983
Epoch 3/20
486/486 [==============================] - 9s 18ms/step - loss: 0.0597 - val_loss: 0.4690
Epoch 4/20
486/486 [==============================] - 9s 18ms/step - loss: 0.0593 - val_loss: 0.4574
Epoch 5/20
486/486 [==============================] - 9s 18ms/step - loss: 0.0592 - val_loss: 0.4353
Epoch 6/20
486/486 [==============================] - 9s 18ms/step - loss: 0.0591 - val_loss: 0.4323
Epoch 7/20
486/486 [==============================] - 9s 18ms/step - loss: 0.0590 - val_loss: 0.4107
Epoch 8/20
486/486 [==============================] - 9s 18ms/step - loss: 0.0590 - val_loss: 0.4075
Epoch 9/20
486/486 [==============================] - 9s 18ms/step - loss: 0.0589 - val_loss: 0.4077
Epoch 10/20
486/486 [==============================] - 9s 19ms/step - loss: 0.0589 - val_l

In [ ]:
# creating the encoder model of the denoising autoencoder
# for the raw, normalized and rescaled data

encoder2_raw = denoising_autoencoder(x_train, x_test)
encoder2_norm = denoising_autoencoder(x_train_norm, x_test_norm)
encoder2_resc = denoising_autoencoder(x_train_resc, x_test_resc)

Epoch 1/20
486/486 [==============================] - 10s 19ms/step - loss: 0.4758 - val_loss: 0.4541
Epoch 2/20
486/486 [==============================] - 9s 18ms/step - loss: 0.4285 - val_loss: 0.4512
Epoch 3/20
486/486 [==============================] - 9s 18ms/step - loss: 0.4255 - val_loss: 0.4488
Epoch 4/20
486/486 [==============================] - 9s 18ms/step - loss: 0.4235 - val_loss: 0.4473
Epoch 5/20
486/486 [==============================] - 9s 19ms/step - loss: 0.4220 - val_loss: 0.4457
Epoch 6/20
486/486 [==============================] - 9s 18ms/step - loss: 0.4211 - val_loss: 0.4443
Epoch 7/20
486/486 [==============================] - 9s 18ms/step - loss: 0.4204 - val_loss: 0.4430
Epoch 8/20
486/486 [==============================] - 9s 19ms/step - loss: 0.4194 - val_loss: 0.4418
Epoch 9/20
486/486 [==============================] - 9s 18ms/step - loss: 0.4189 - val_loss: 0.4409
Epoch 10/20
486/486 [==============================] - 9s 18ms/step - loss: 0.4186 - val_l

In [ ]:
# creating the encoder model of sparse autoencoder
# for the raw, normalized and rescaled data
encoder3_raw = sparse_autoencoder(x_train, x_test)
encoder3_norm = sparse_autoencoder(x_train_norm, x_test_norm)
encoder3_resc = sparse_autoencoder(x_train_resc, x_test_resc) 

Epoch 1/20
486/486 [==============================] - 2s 3ms/step - loss: 0.3376 - val_loss: 0.2651
Epoch 2/20
486/486 [==============================] - 1s 2ms/step - loss: 0.0778 - val_loss: 0.2791
Epoch 3/20
486/486 [==============================] - 1s 2ms/step - loss: 0.0699 - val_loss: 0.2842
Epoch 4/20
486/486 [==============================] - 1s 3ms/step - loss: 0.0661 - val_loss: 0.2885
Epoch 5/20
486/486 [==============================] - 1s 2ms/step - loss: 0.0639 - val_loss: 0.2920
Epoch 6/20
486/486 [==============================] - 1s 2ms/step - loss: 0.0627 - val_loss: 0.2962
Epoch 7/20
486/486 [==============================] - 1s 2ms/step - loss: 0.0621 - val_loss: 0.3006
Epoch 8/20
486/486 [==============================] - 1s 2ms/step - loss: 0.0616 - val_loss: 0.3040
Epoch 9/20
486/486 [==============================] - 1s 2ms/step - loss: 0.0613 - val_loss: 0.3066
Epoch 10/20
486/486 [==============================] - 1s 2ms/step - loss: 0.0609 - val_loss: 0.3072

In [ ]:
# data transformation - encoder1
x_train_enc_1, x_test_enc_1 = encoder_transformation(encoder1_raw, x_train, x_test)
x_train_norm_enc_1, x_test_norm_enc_1 = encoder_transformation(encoder1_norm, x_train_norm, x_test_norm) 
x_train_resc_enc_1, x_test_resc_enc_1 = encoder_transformation(encoder1_resc, x_train_resc, x_test_resc)

In [ ]:
# data transformation - encoder2
x_train_enc_2, x_test_enc_2 = encoder_transformation(encoder2_raw, x_train, x_test)
x_train_norm_enc_2, x_test_norm_enc_2 = encoder_transformation(encoder2_norm, x_train_norm, x_test_norm) 
x_train_resc_enc_2, x_test_resc_enc_2 = encoder_transformation(encoder2_resc, x_train_resc, x_test_resc)

In [ ]:
# data transformation - encoder3
x_train_enc_3, x_test_enc_3 = encoder_transformation(encoder3_raw, x_train, x_test)
x_train_norm_enc_3, x_test_norm_enc_3 = encoder_transformation(encoder3_norm, x_train_norm, x_test_norm) 
x_train_resc_enc_3, x_test_resc_enc_3 = encoder_transformation(encoder3_resc, x_train_resc, x_test_resc)

FITTING MODELS TO EXTRACTED DATA

In [ ]:
# encoder1 - raw data
compare_algorithms(x_train_enc_1, y_train, x_test_enc_1, y_test)

LR: train_acc: 0.990 / test_acc: 0.516
LDA: train_acc: 0.981 / test_acc: 0.698
KNN: train_acc: 1.000 / test_acc: 0.534
SVC: train_acc: 0.998 / test_acc: 0.534
LnSVC: train_acc: 0.991 / test_acc: 0.518


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder1 - normalized data
compare_algorithms(x_train_norm_enc_1, y_train, x_test_norm_enc_1, y_test)

LR: train_acc: 0.993 / test_acc: 0.821
LDA: train_acc: 0.979 / test_acc: 0.929
KNN: train_acc: 1.000 / test_acc: 0.529
SVC: train_acc: 0.997 / test_acc: 0.505
LnSVC: train_acc: 0.994 / test_acc: 0.750


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder1 - rescaled data
compare_algorithms(x_train_resc_enc_1, y_train, x_test_resc_enc_1, y_test)

LR: train_acc: 0.955 / test_acc: 0.851
LDA: train_acc: 0.946 / test_acc: 0.905
KNN: train_acc: 1.000 / test_acc: 0.500
SVC: train_acc: 0.999 / test_acc: 0.500
LnSVC: train_acc: 0.954 / test_acc: 0.854


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder2 - raw data
compare_algorithms(x_train_enc_2, y_train, x_test_enc_2, y_test)

LR: train_acc: 0.988 / test_acc: 0.522
LDA: train_acc: 0.976 / test_acc: 0.554
KNN: train_acc: 0.999 / test_acc: 0.521
SVC: train_acc: 0.996 / test_acc: 0.525
LnSVC: train_acc: 0.990 / test_acc: 0.528


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder2 - normalized data
compare_algorithms(x_train_norm_enc_2, y_train, x_test_norm_enc_2, y_test)

LR: train_acc: 0.990 / test_acc: 0.500
LDA: train_acc: 0.975 / test_acc: 0.500
KNN: train_acc: 0.999 / test_acc: 0.500
SVC: train_acc: 0.992 / test_acc: 0.500
LnSVC: train_acc: 0.993 / test_acc: 0.500


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder2 - rescaled data
compare_algorithms(x_train_resc_enc_2, y_train, x_test_resc_enc_2, y_test)

LR: train_acc: 0.993 / test_acc: 0.907
LDA: train_acc: 0.979 / test_acc: 0.906
KNN: train_acc: 1.000 / test_acc: 0.499
SVC: train_acc: 0.998 / test_acc: 0.499
LnSVC: train_acc: 0.993 / test_acc: 0.908


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder3 - raw data
compare_algorithms(x_train_enc_3, y_train, x_test_enc_3, y_test)

LR: train_acc: 0.990 / test_acc: 0.530
LDA: train_acc: 0.976 / test_acc: 0.916
KNN: train_acc: 0.999 / test_acc: 0.501
SVC: train_acc: 0.997 / test_acc: 0.532
LnSVC: train_acc: 0.992 / test_acc: 0.525


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder3 - normalized data
compare_algorithms(x_train_norm_enc_3, y_train, x_test_norm_enc_3, y_test)

LR: train_acc: 0.990 / test_acc: 0.639
LDA: train_acc: 0.980 / test_acc: 0.808
KNN: train_acc: 0.999 / test_acc: 0.528
SVC: train_acc: 0.995 / test_acc: 0.527
LnSVC: train_acc: 0.992 / test_acc: 0.962


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# encoder3 - rescaled data
compare_algorithms(x_train_resc_enc_3, y_train, x_test_resc_enc_3, y_test)

LR: train_acc: 0.994 / test_acc: 0.460
LDA: train_acc: 0.978 / test_acc: 0.905
KNN: train_acc: 1.000 / test_acc: 0.500
SVC: train_acc: 0.998 / test_acc: 0.499
LnSVC: train_acc: 0.995 / test_acc: 0.448


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


COMBINE ORIGINAL WITH EXTRACTED FEATURES (NORMALIZED DATA, ENCODERS 1 AND 3)



In [ ]:
# FULL SET OF ORIGINAL FEATURES + EXTRACTED FEATURES
# 82 + 20 = 102 FEATURES 

In [ ]:
# stacked encoder
x_train_full_enc_1, x_test_full_enc_1 = mixed_data(x_train_norm, x_train_norm_enc_1, x_test_norm, x_test_norm_enc_1)
# sparse encoder
x_train_full_enc_3, x_test_full_enc_3 = mixed_data(x_train_norm, x_train_norm_enc_3, x_test_norm, x_test_norm_enc_3)

In [ ]:
# SELECTED SET OF ORIGINAL FEATURES (EXTRA TREES CLASSIFIER) + EXTRACTED FEATURES
# 10 + 20 = 30 FEATURES

In [ ]:
# stacked encoder
x_train_sel_enc_1, x_test_sel_enc_1 = mixed_data(extree_train_features_norm, x_train_norm_enc_1, extree_test_features_norm, x_test_norm_enc_1)
# sparse encoder
x_train_sel_enc_3, x_test_sel_enc_3 = mixed_data(extree_train_features_norm, x_train_norm_enc_3, extree_test_features_norm, x_test_norm_enc_3)

COMBINE ORIGINAL WITH EXTRACTED FEATURES (NORMALIZED DATA, ENCODERS 1 AND 3) - MODEL EVALUATION (LR, LDA, LnSVC)

In [ ]:
# FULL SET OF ORIGINAL FEATURES + EXTRACTED FEATURES
# 82 + 20 = 102 FEATURES 

In [ ]:
# stacked encoder
compare_algorithms_2(x_train_full_enc_1, y_train, x_test_full_enc_1, y_test)

LR:
[[19503   576]
 [ 6181 13898]]
acc:  0.832  / DR:  0.692  / Prec:  0.96  / FAR:  0.029  / FNR:  0.308
Type I error:  576  / Type II error:  6181
TBM:  2.13
TTM:  0.12 

LDA:
[[16750  3329]
 [12871  7208]]
acc:  0.597  / DR:  0.359  / Prec:  0.684  / FAR:  0.166  / FNR:  0.641
Type I error:  3329  / Type II error:  12871
TBM:  2.79
TTM:  0.12 

LnSVC:
[[19187   892]
 [ 7680 12399]]
acc:  0.787  / DR:  0.618  / Prec:  0.933  / FAR:  0.044  / FNR:  0.382
Type I error:  892  / Type II error:  7680
TBM:  14.56
TTM:  0.12 



/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# sparse encoder
compare_algorithms_2(x_train_full_enc_3, y_train, x_test_full_enc_3, y_test)

LR:
[[19190   889]
 [ 4797 15282]]
acc:  0.858  / DR:  0.761  / Prec:  0.945  / FAR:  0.044  / FNR:  0.239
Type I error:  889  / Type II error:  4797
TBM:  1.92
TTM:  0.12 

LDA:
[[   31 20048]
 [    0 20079]]
acc:  0.501  / DR:  1.0  / Prec:  0.5  / FAR:  0.998  / FNR:  0.0
Type I error:  20048  / Type II error:  0
TBM:  2.6
TTM:  0.15 

LnSVC:
[[19341   738]
 [12555  7524]]
acc:  0.669  / DR:  0.375  / Prec:  0.911  / FAR:  0.037  / FNR:  0.625
Type I error:  738  / Type II error:  12555
TBM:  9.26
TTM:  0.12 



/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# SELECTED SET OF ORIGINAL FEATURES (EXTRA TREES CLASSIFIER) + EXTRACTED FEATURES
# 10 + 20 = 30 FEATURES

In [ ]:
# stacked encoder
compare_algorithms_2(x_train_sel_enc_1, y_train, x_test_sel_enc_1, y_test)

LR:
[[19521   558]
 [ 6729 13350]]
acc:  0.819  / DR:  0.665  / Prec:  0.96  / FAR:  0.028  / FNR:  0.335
Type I error:  558  / Type II error:  6729
TBM:  0.96
TTM:  0.11 

LDA:
[[18594  1485]
 [    6 20073]]
acc:  0.963  / DR:  1.0  / Prec:  0.931  / FAR:  0.074  / FNR:  0.0
Type I error:  1485  / Type II error:  6
TBM:  0.47
TTM:  0.12 

LnSVC:
[[19612   467]
 [18551  1528]]
acc:  0.526  / DR:  0.076  / Prec:  0.766  / FAR:  0.023  / FNR:  0.924
Type I error:  467  / Type II error:  18551
TBM:  8.41
TTM:  0.11 



/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
# sparse encoder
compare_algorithms_2(x_train_sel_enc_3, y_train, x_test_sel_enc_3, y_test)

LR:
[[19167   912]
 [ 8481 11598]]
acc:  0.766  / DR:  0.578  / Prec:  0.927  / FAR:  0.045  / FNR:  0.422
Type I error:  912  / Type II error:  8481
TBM:  1.01
TTM:  0.11 

LDA:
[[16187  3892]
 [    2 20077]]
acc:  0.903  / DR:  1.0  / Prec:  0.838  / FAR:  0.194  / FNR:  0.0
Type I error:  3892  / Type II error:  2
TBM:  0.49
TTM:  0.11 

LnSVC:
[[18879  1200]
 [ 5561 14518]]
acc:  0.832  / DR:  0.723  / Prec:  0.924  / FAR:  0.06  / FNR:  0.277
Type I error:  1200  / Type II error:  5561
TBM:  6.21
TTM:  0.12 



/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


FEATURE SELECTION (EXTRA TREES CLASSIFIER) ON COMBINATION OF ORIGINAL WITH EXTRACTED FEATURES (NORMALIZED DATA)


In [ ]:
# SELECTED 10 FEATURES FROM THE FULL SET OF ORIGINAL FEATURES + EXTRACTED FEATURES

In [ ]:
# stacked encoder
sel_x_train_full_enc_1, sel_x_test_full_enc_1 = extra_trees_classifier(x_train_full_enc_1, y_train, x_test_full_enc_1)

# sparse encoder
sel_x_train_full_enc_3, sel_x_test_full_enc_3 = extra_trees_classifier(x_train_full_enc_3, y_train, x_test_full_enc_3)

    Specs     Score
26     26  0.094312
92     92  0.073974
15     15  0.059655
22     22  0.055845
13     13  0.055475
16     16  0.042157
28     28  0.040564
36     36  0.036718
2       2  0.033981
94     94  0.033788
    Specs     Score
28     28  0.087085
26     26  0.078875
22     22  0.065940
15     15  0.050983
23     23  0.044327
13     13  0.043780
16     16  0.040299
21     21  0.033958
91     91  0.029929
85     85  0.026591
    Specs     Score
26     26  0.097576
22     22  0.067936
15     15  0.060225
86     86  0.049294
28     28  0.044003
23     23  0.042379
13     13  0.037628
16     16  0.034602
82     82  0.027657
95     95  0.027466


In [ ]:
# SELECTED 10 FEATURES FROM THE SELECTED SET OF ORIGINAL FEATURES + EXTRACTED FEATURES

In [ ]:
# stacked encoder
sel_x_train_sel_enc_1, sel_x_test_sel_enc_1 = extra_trees_classifier(x_train_sel_enc_1, y_train, x_test_sel_enc_1)

# sparse encoder
sel_x_train_sel_enc_3, sel_x_test_sel_enc_3 = extra_trees_classifier(x_train_sel_enc_3, y_train, x_test_sel_enc_3)

    Specs     Score
1       1  0.095942
2       2  0.094538
20     20  0.078641
11     11  0.076186
8       8  0.064663
0       0  0.064064
3       3  0.061144
5       5  0.060813
21     21  0.049861
4       4  0.047966
    Specs     Score
2       2  0.107647
13     13  0.076222
0       0  0.073013
8       8  0.072921
1       1  0.070803
15     15  0.050642
19     19  0.046028
11     11  0.045331
5       5  0.044961
3       3  0.043231
    Specs     Score
2       2  0.097329
8       8  0.092893
10     10  0.076735
5       5  0.069998
1       1  0.068867
3       3  0.068590
14     14  0.066105
0       0  0.062925
23     23  0.045937
18     18  0.043053


FEATURE SELECTION (EXTRA TREES CLASSIFIER) ON COMBINATION OF ORIGINAL WITH EXTRACTED FEATURES (NORMALIZED DATA) - MODEL EVALUATION (LR, LDA, LnSVC)

In [ ]:
# SELECTED 10 FEATURES FROM THE FULL SET OF ORIGINAL FEATURES + EXTRACTED 
# 82 + 20 --> 10

In [ ]:
# stacked encoder
compare_algorithms_2(sel_x_train_full_enc_1, y_train, sel_x_test_full_enc_1, y_test)

LR:
[[19157   922]
 [ 4781 15298]]
acc:  0.858  / DR:  0.762  / Prec:  0.943  / FAR:  0.046  / FNR:  0.238
Type I error:  922  / Type II error:  4781
TBM:  0.29
TTM:  0.11 

LDA:
[[18310  1769]
 [   42 20037]]
acc:  0.955  / DR:  0.998  / Prec:  0.919  / FAR:  0.088  / FNR:  0.002
Type I error:  1769  / Type II error:  42
TBM:  0.14
TTM:  0.15 

LnSVC:
[[19318   761]
 [ 4741 15338]]
acc:  0.863  / DR:  0.764  / Prec:  0.953  / FAR:  0.038  / FNR:  0.236
Type I error:  761  / Type II error:  4741
TBM:  2.81
TTM:  0.12 



In [ ]:
# sparse encoder
compare_algorithms_2(sel_x_train_full_enc_3, y_train, sel_x_test_full_enc_3, y_test)

LR:
[[17822  2257]
 [ 1452 18627]]
acc:  0.908  / DR:  0.928  / Prec:  0.892  / FAR:  0.112  / FNR:  0.072
Type I error:  2257  / Type II error:  1452
TBM:  0.39
TTM:  0.12 

LDA:
[[16801  3278]
 [ 1466 18613]]
acc:  0.882  / DR:  0.927  / Prec:  0.85  / FAR:  0.163  / FNR:  0.073
Type I error:  3278  / Type II error:  1466
TBM:  0.15
TTM:  0.15 

LnSVC:
[[18868  1211]
 [ 1444 18635]]
acc:  0.934  / DR:  0.928  / Prec:  0.939  / FAR:  0.06  / FNR:  0.072
Type I error:  1211  / Type II error:  1444
TBM:  4.37
TTM:  0.11 



In [ ]:
# SELECTED 10 FEATURES FROM THE SELECTED SET OF ORIGINAL FEATURES + EXTRACTED FEATURES
# 10 + 20 --> 10

In [ ]:
# stacked encoder
compare_algorithms_2(sel_x_train_sel_enc_1, y_train, sel_x_test_sel_enc_1, y_test)

LR:
[[17574  2505]
 [   28 20051]]
acc:  0.937  / DR:  0.999  / Prec:  0.889  / FAR:  0.125  / FNR:  0.001
Type I error:  2505  / Type II error:  28
TBM:  0.32
TTM:  0.12 

LDA:
[[16800  3279]
 [    5 20074]]
acc:  0.918  / DR:  1.0  / Prec:  0.86  / FAR:  0.163  / FNR:  0.0
Type I error:  3279  / Type II error:  5
TBM:  0.15
TTM:  0.15 

LnSVC:
[[18434  1645]
 [   47 20032]]
acc:  0.958  / DR:  0.998  / Prec:  0.924  / FAR:  0.082  / FNR:  0.002
Type I error:  1645  / Type II error:  47
TBM:  2.69
TTM:  0.11 



In [ ]:
# sparse encoder
compare_algorithms_2(sel_x_train_sel_enc_3, y_train, sel_x_test_sel_enc_2, y_test)

LR:
[[    0 20079]
 [    0 20079]]
acc:  0.5  / DR:  1.0  / Prec:  0.5  / FAR:  1.0  / FNR:  0.0
Type I error:  20079  / Type II error:  0
TBM:  0.41
TTM:  0.12 

LDA:
[[    0 20079]
 [    0 20079]]
acc:  0.5  / DR:  1.0  / Prec:  0.5  / FAR:  1.0  / FNR:  0.0
Type I error:  20079  / Type II error:  0
TBM:  0.15
TTM:  0.12 

LnSVC:
[[    0 20079]
 [    0 20079]]
acc:  0.5  / DR:  1.0  / Prec:  0.5  / FAR:  1.0  / FNR:  0.0
Type I error:  20079  / Type II error:  0
TBM:  5.3
TTM:  0.11 



/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


NEURAL NETWORK MODEL

In [ ]:
# use a reduced (train) data set for grid search 
reduced_x_train, reduced_y_train = reduced_data(df_train_data_filtered, 2000)

In [ ]:
grid_search(reduced_x_train, reduced_y_train)
"""Best: 0.471667 using {'batch_size': 5, 'epochs': 5, 'init': 'glorot_uniform', 'optimizer': 'adam'}"""

In [ ]:
def create_nn_model(neurons, number_of_features, init="glorot_uniform"):
  # returns a fully-connected network structure with three layers.

  model=Sequential()
  model.add(Dense(neurons, input_dim=number_of_features, activation="relu", kernel_initializer=init))
  model.add(Dropout(0.3))
  model.add(Dense(1, activation="softmax", kernel_initializer=init))

  model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

  return model

In [ ]:
start1 = time.time()

model = create_nn_model(82)
model.fit(x_train, y_train, batch_size=5, epochs=5)

train_time = time.time()

loss, accuracy = model.evaluate(x_train, y_train)

# predictions
start2 = time.time()

predictions = model.predict(x_test)
test_acc = accuracy_score(y_test, predictions)
test_auc = roc_auc_score(y_test, predictions)
class_report = classification_report(y_test, predictions)

test_time = time.time()

print("acc: %.3f" % (test_acc))
print("auc: %.3f" % (test_auc))
print("TBM: ", train_time-start, "seconds")
print("TTM:", test_time-train_time, "seconds")
print(class_report)

Epoch 1/5
19409/19409 [==============================] - 68s 3ms/step - loss: 0.0565 - accuracy: 0.4980
Epoch 2/5
19409/19409 [==============================] - 66s 3ms/step - loss: 0.0332 - accuracy: 0.5035
Epoch 3/5
19409/19409 [==============================] - 66s 3ms/step - loss: 0.0339 - accuracy: 0.4986
Epoch 4/5
19409/19409 [==============================] - 67s 3ms/step - loss: 0.0318 - accuracy: 0.5014
Epoch 5/5
3033/3033 [==============================] - 5s 1ms/step - loss: 0.0186 - accuracy: 0.5000


In [ ]:
# Grid search for number of neurons of the NN

def grid_search_neurons(features, input, output):

  from keras.constraints import maxnorm
 
  # Function to create model, required for KerasClassifier
  def create_model(neurons=1):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=features, activation="relu", kernel_initializer="glorot_uniform"))
    model.add(Dense(5, kernel_initializer="glorot_uniform", activation="relu"))
    model.add(Dense(1, kernel_initializer="glorot_uniform", activation="softmax"))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return model

  model = KerasClassifier(build_fn=create_model, epochs=40, batch_size=5, verbose=0)
  # define the grid search parameters
  neurons = [1,5,7]
  param_grid = dict(neurons=neurons)
  grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
  grid_result = grid.fit(input, output)
  # summarize results
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  for mean, stdev, param in zip(means, stds, params):
      print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
# Grid search for epochs, batch size, optimizer, initializer of the NN

def grid_search(features, input, output):

  # create model

  def create_model(optimizer="adam", init="glorot_uniform"):
    model=Sequential()
    model.add(Dense(103, input_dim=features, kernel_initializer=init, activation="relu"))
    model.add(Dense(60, kernel_initializer=init, activation="relu"))
    model.add(Dense(1, kernel_initializer=init, activation="softmax"))

    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model
  model = KerasClassifier(build_fn = create_model, verbose=0)

  # grid search to find epochs, batch_size, optimizer
  epochs_list = [40,50]
  batches_list = [5,15]
  #optimizer_list = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
  optimizer_list = ["SGD"]
  #inits_list = ["glorot_uniform", "normal", "uniform"]
  inits_list = ["glorot_uniform"]

  param_grid_dictionary=dict(optimizer=optimizer_list, epochs=epochs_list, batch_size=batches_list, init=inits_list)
  type(param_grid_dictionary)
  grid=GridSearchCV(estimator=model,param_grid=param_grid_dictionary)
  grid_result=grid.fit(input,output)

  # summarize results
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
xy_train_full_enc_1 = mixed_data_2(x_train_full_enc_1, y_train)
reduced_x_train_full_enc_1, reduced_y_train_full_enc_1 = reduced_data(xy_train_full_enc_1, 600)

In [ ]:
grid_search(102, reduced_x_train_full_enc_1, reduced_y_train_full_enc_1)

Best: 0.471667 using {'batch_size': 5, 'epochs': 40, 'init': 'glorot_uniform', 'optimizer': 'SGD'}
0.471667 (0.031447) with: {'batch_size': 5, 'epochs': 40, 'init': 'glorot_uniform', 'optimizer': 'SGD'}
0.471667 (0.031447) with: {'batch_size': 5, 'epochs': 50, 'init': 'glorot_uniform', 'optimizer': 'SGD'}
0.471667 (0.031447) with: {'batch_size': 15, 'epochs': 40, 'init': 'glorot_uniform', 'optimizer': 'SGD'}
0.471667 (0.031447) with: {'batch_size': 15, 'epochs': 50, 'init': 'glorot_uniform', 'optimizer': 'SGD'}


In [ ]:
grid_search_neurons(102, reduced_x_train_full_enc_1, reduced_y_train_full_enc_1)

Best: 0.471667 using {'neurons': 1}
0.471667 (0.006236) with: {'neurons': 1}
0.471667 (0.006236) with: {'neurons': 5}
0.471667 (0.006236) with: {'neurons': 7}
